In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%cd /home/ubuntu/projects/hyper-sloth

In [2]:
from speedy_utils.all import *
from llm_utils import *
from fastcore.all import *



#### Create a LLM model

In [3]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Choose any GPU you want to use
from vllm import LLM, SamplingParams
import torch

INFO 03-15 08:13:26 [__init__.py:256] Automatically detected platform cuda.


In [4]:
# setup logger
import logging

if "llm" in locals():
    del llm  #
    torch.cuda.empty_cache()
    
llm = LLM(
    model="google/gemma-3-27b-it",
    tensor_parallel_size=1,
    task="generate",
    enforce_eager=True,
    dtype=torch.bfloat16,
    max_model_len=1024,
    quantization="bitsandbytes", load_format="bitsandbytes"
)

tokenizer = llm.get_tokenizer()

INFO 03-15 08:13:29 [config.py:2579] Downcasting torch.float32 to torch.bfloat16.
WARNING 03-15 08:13:37 [config.py:662] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 03-15 08:13:37 [arg_utils.py:1759] --quantization bitsandbytes is not supported by the V1 Engine. Falling back to V0. 
WARNING 03-15 08:13:37 [cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-15 08:13:38 [llm_engine.py:241] Initializing a V0 LLM engine (v0.7.4.dev473+g9ed6ee92) with config: model='google/gemma-3-27b-it', speculative_config=None, tokenizer='google/gemma-3-27b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_paralle

Loading safetensors checkpoint shards:   0% Completed | 0/12 [00:00<?, ?it/s]


INFO 03-15 08:13:53 [model_runner.py:1146] Model loading took 15.5312 GB and 12.066275 seconds


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO 03-15 08:14:08 [worker.py:267] Memory profiling takes 14.58 seconds
INFO 03-15 08:14:08 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.64GiB) x gpu_memory_utilization (0.90) = 21.28GiB
INFO 03-15 08:14:08 [worker.py:267] model weights take 15.53GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.19GiB; the rest of the memory reserved for KV Cache is 3.48GiB.
INFO 03-15 08:14:09 [executor_base.py:111] # cuda blocks: 459, # CPU blocks: 528
INFO 03-15 08:14:09 [executor_base.py:116] Maximum concurrency for 1024 tokens per request: 7.17x
INFO 03-15 08:14:12 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 18.44 seconds


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
data = load_by_ext(
    "/home/ubuntu/projects/localization/data/sharegpt/LC_GS1_JX1M_VI_test.json"
)
msgs = data[5]["messages"]
prompt = tokenizer.apply_chat_template(
    msgs[:-1], tokenize=False, add_generation_prompt=True
) + 'Translation: '
sampling_params = SamplingParams(temperature=0.2, top_p=0.95, top_k=64, max_tokens=50)
outputs = llm.generate([prompt], sampling_params)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	

#### GT
`'%</color> tỉ lệ thi triển <color=#1560b5>『Lệ Hồn Đoạt Phách』</color>, thời gian chờ <color=#d15e00>'`
##### 4B output
`"Có xác suất tự động kích hoạt <color=#1560b5>『Lì Hồn Dấu Phách』</color>, thời gian hồi chiên <color=#d15e00>"`

In [6]:
print(outputs[0].outputs[0].text)



**%</color> xác suất kích hoạt tự động <color=#1560b5>『Lệ Hồn Đoạt Phách』</color>, thời gian hồi chiêu <color=#d15e00>**


In [7]:
display_chat_messages_as_html(msgs)